## 导入包

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import datetime
import time
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectPercentile,f_regression
pd.options.display.max_columns = None  # 显示的最大列数不受限制

## 读取数据

In [2]:
train_1 = pd.read_csv('./data2/train/train_1.csv')
train_2 = pd.read_csv('./data2/train/train_2.csv')
train_3 = pd.read_csv('./data2/train/train_3.csv')
train_4 = pd.read_csv('./data2/train/train_4.csv')
train_5 = pd.read_csv('./data2/train/train_5.csv')
train_6 = pd.read_csv('./data2/train/train_6.csv')
train_7 = pd.read_csv('./data2/train/train_7.csv')
train_8 = pd.read_csv('./data2/train/train_8.csv')
train_9 = pd.read_csv('./data2/train/train_9.csv')
train_10 = pd.read_csv('./data2/train/train_10.csv')

test_1 = pd.read_csv('./data2/test/test_1.csv')
test_2 = pd.read_csv('./data2/test/test_2.csv')
test_3 = pd.read_csv('./data2/test/test_3.csv')
test_4 = pd.read_csv('./data2/test/test_4.csv')
test_5 = pd.read_csv('./data2/test/test_5.csv')
test_6 = pd.read_csv('./data2/test/test_6.csv')
test_7 = pd.read_csv('./data2/test/test_7.csv')
test_8 = pd.read_csv('./data2/test/test_8.csv')
test_9 = pd.read_csv('./data2/test/test_9.csv')
test_10 = pd.read_csv('./data2/test/test_10.csv')


In [3]:
data_1 = pd.concat([train_1,test_1],axis=0,ignore_index=True,sort=False)
data_2 = pd.concat([train_2,test_2],axis=0,ignore_index=True,sort=False)
data_3 = pd.concat([train_3,test_3],axis=0,ignore_index=True,sort=False)
data_4 = pd.concat([train_4,test_4],axis=0,ignore_index=True,sort=False)
data_5 = pd.concat([train_5,test_5],axis=0,ignore_index=True,sort=False)
data_6 = pd.concat([train_6,test_6],axis=0,ignore_index=True,sort=False)
data_7 = pd.concat([train_7,test_7],axis=0,ignore_index=True,sort=False)
data_8 = pd.concat([train_8,test_8],axis=0,ignore_index=True,sort=False)
data_9 = pd.concat([train_9,test_9],axis=0,ignore_index=True,sort=False)
data_10 = pd.concat([train_10,test_10],axis=0,ignore_index=True,sort=False)

In [4]:
# f1 = pd.DataFrame(data_4.groupby("风速")["风向"].count()).reset_index().rename(columns={"风向":"count of 风速 and 风向"})
# f2 = pd.DataFrame(data_4.groupby("风速")["风向"].mean()).reset_index().rename(columns={"风向":"mean of 风速 and 风向"})
# f3 = pd.DataFrame(data_4.groupby("风速")["风向"].nunique()).reset_index().rename(columns={"风向":"nuq of 风速 and 风向"})
# f4 = pd.DataFrame(data_4.groupby("湿度")["风向"].mean()).reset_index().rename(columns={"风向":"mean of 湿度 and 风向"})
# f5 = pd.DataFrame(data_4.groupby("风向")["风速"].mean()).reset_index().rename(columns={"风速":"mean of 风向 and 风速"})

In [5]:
# data_4 = pd.merge(data_4,f1, how='left', on=["风速"])
# data_4 = pd.merge(data_4,f2, how='left', on=["风速"])
# data_4 = pd.merge(data_4,f3, how='left', on=["风速"])
# data_4 = pd.merge(data_4,f4, how='left', on=["湿度"])
# data_4 = pd.merge(data_4,f5, how='left', on=["风向"])

In [6]:
# # 返回时间戳
# def get_timestamp(x):
#     try:
#         t = datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S.%f")
#         t = int(round((t.timestamp())))  # 四舍五入，去掉毫秒级  
#     except:
#         t = datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S")
#         t = int(t.timestamp())
#     return t
    

In [7]:
data_1.head()

,时间,辐照度,风速,风向,温度,湿度,压强,实际辐照度,实际功率,id
0,2017-01-01 00:15:00,-1.0,-0.855915,0.291529,-0.459561,-0.207037,0.043513,0.0,0.039000,NaN
1,2017-01-01 00:30:00,-1.0,-0.851950,0.395271,-0.462905,-0.200263,0.042722,0.0,0.039000,NaN
2,2017-01-01 00:45:00,-1.0,-0.847984,0.497236,-0.465831,-0.192790,0.041930,0.0,0.042333,NaN
3,2017-01-01 01:00:00,-1.0,-0.844019,0.592254,-0.469592,-0.184763,0.041139,0.0,0.042667,NaN
4,2017-01-01 01:15:00,-1.0,-0.838731,0.675103,-0.473354,-0.176346,0.040348,0.0,0.042667,NaN


In [8]:
data_1.dtypes

时间        object
辐照度      float64
风速       float64
风向       float64
温度       float64
湿度       float64
压强       float64
实际辐照度    float64
实际功率     float64
id       float64
dtype: object

In [9]:
type(data_1['时间'])

pandas.core.series.Series

In [10]:
data_1['时间戳'] = pd.to_datetime(data_1['时间'])
data = data_1[['时间戳','辐照度','风速','风向','温度','压强','湿度']]
data = data.drop_duplicates(['时间戳'])  # 去掉重复值，不去掉合并会多行

In [11]:
t1 = data.copy()

In [12]:
t1.head()

,时间戳,辐照度,风速,风向,温度,压强,湿度
0,2017-01-01 00:15:00,-1.0,-0.855915,0.291529,-0.459561,0.043513,-0.207037
1,2017-01-01 00:30:00,-1.0,-0.851950,0.395271,-0.462905,0.042722,-0.200263
2,2017-01-01 00:45:00,-1.0,-0.847984,0.497236,-0.465831,0.041930,-0.192790
3,2017-01-01 01:00:00,-1.0,-0.844019,0.592254,-0.469592,0.041139,-0.184763
4,2017-01-01 01:15:00,-1.0,-0.838731,0.675103,-0.473354,0.040348,-0.176346


In [13]:
t1.columns = ['时间戳','辐照度-1','风速-1','风向-1','温度-1','压强-1','湿度-1']

In [14]:
t1.head()

,时间戳,辐照度-1,风速-1,风向-1,温度-1,压强-1,湿度-1
0,2017-01-01 00:15:00,-1.0,-0.855915,0.291529,-0.459561,0.043513,-0.207037
1,2017-01-01 00:30:00,-1.0,-0.851950,0.395271,-0.462905,0.042722,-0.200263
2,2017-01-01 00:45:00,-1.0,-0.847984,0.497236,-0.465831,0.041930,-0.192790
3,2017-01-01 01:00:00,-1.0,-0.844019,0.592254,-0.469592,0.041139,-0.184763
4,2017-01-01 01:15:00,-1.0,-0.838731,0.675103,-0.473354,0.040348,-0.176346


In [15]:
# t1 = data.copy()
# t1.columns = ['时间戳','辐照度-1','风速-1','风向-1','温度-1','压强-1','湿度-1']
t1_0 = t1[:1].copy() # 获取第一行，用于合并

In [16]:

t1['时间戳'] = t1['时间戳'].map(lambda x: x+pd.Timedelta(seconds=900))  #15分*60=900秒
t1 = pd.concat([t1_0,t1],axis=0)
data_1 = pd.merge(data_1,t1,how='left',on='时间戳')

In [17]:
data_1.head()

,时间,辐照度,风速,风向,温度,湿度,压强,实际辐照度,实际功率,id,时间戳,辐照度-1,风速-1,风向-1,温度-1,压强-1,湿度-1
0,2017-01-01 00:15:00,-1.0,-0.855915,0.291529,-0.459561,-0.207037,0.043513,0.0,0.039000,NaN,2017-01-01 00:15:00,-1.0,-0.855915,0.291529,-0.459561,0.043513,-0.207037
1,2017-01-01 00:30:00,-1.0,-0.851950,0.395271,-0.462905,-0.200263,0.042722,0.0,0.039000,NaN,2017-01-01 00:30:00,-1.0,-0.855915,0.291529,-0.459561,0.043513,-0.207037
2,2017-01-01 00:45:00,-1.0,-0.847984,0.497236,-0.465831,-0.192790,0.041930,0.0,0.042333,NaN,2017-01-01 00:45:00,-1.0,-0.851950,0.395271,-0.462905,0.042722,-0.200263
3,2017-01-01 01:00:00,-1.0,-0.844019,0.592254,-0.469592,-0.184763,0.041139,0.0,0.042667,NaN,2017-01-01 01:00:00,-1.0,-0.847984,0.497236,-0.465831,0.041930,-0.192790
4,2017-01-01 01:15:00,-1.0,-0.838731,0.675103,-0.473354,-0.176346,0.040348,0.0,0.042667,NaN,2017-01-01 01:15:00,-1.0,-0.844019,0.592254,-0.469592,0.041139,-0.184763


In [18]:
t2 = data.copy()
t2.columns = ['时间戳','辐照度-2','风速-2','风向-2','温度-2','压强-2','湿度-2']
t2_0 = t2[:2].copy() # 获取第一行，用于合并
t2['时间戳'] = t2['时间戳'].map(lambda x: x+pd.Timedelta(seconds=1800))  #30分*60=1800秒
t2 = pd.concat([t2_0,t2],axis=0)
data_1 = pd.merge(data_1,t2,how='left',on='时间戳')


t1 = data.copy()
t1.columns = ['时间戳','辐照度+1','风速+1','风向+1','温度+1','压强+1','湿度+1']
t1_0 = t1[-1:].copy() # 获取最后一行，用于合并
t1['时间戳'] = t1['时间戳'].map(lambda x: x-pd.Timedelta(seconds=900))  #15分*60=900秒
t1 = pd.concat([t1,t1_0],axis=0)
data_1 = pd.merge(data_1,t1,how='left',on='时间戳')

t2 = data.copy()
t2.columns = ['时间戳','辐照度+2','风速+2','风向+2','温度+2','压强+2','湿度+2']
t2_0 = t2[-2:].copy() # 获取最后两行，用于合并
t2['时间戳'] = t2['时间戳'].map(lambda x:  x-pd.Timedelta(seconds=1800))  #30分*60=900秒
t2 = pd.concat([t2,t2_0],axis=0)
data_1 = pd.merge(data_1,t2,how='left',on='时间戳')

t3 = data.copy()
t3.columns = ['时间戳','辐照度+3','风速+3','风向+3','温度+3','压强+3','湿度+3']
t3_0 = t3[-3:].copy() # 获取最后三行，用于合并
t3['时间戳'] = t3['时间戳'].map(lambda x: x-pd.Timedelta(seconds=2700))  #45分*60=2700秒
t3 = pd.concat([t3,t3_0],axis=0)
data_1 = pd.merge(data_1,t3,how='left',on='时间戳')

In [19]:
index_list = data_1[data_1['辐照度+2'].isnull()].index.tolist()
for index in index_list:
    for feat in ['辐照度','风速','风向','温度','压强','湿度']:
        data_1.loc[index,feat+'+2'] = data_1.loc[index,feat+'+1']

In [20]:

data_1['rolling_sum3']=data.风速.rolling(3).mean()

In [25]:
data_1['风速_diff1']=data.风速.diff(-1)

In [23]:
data_1[['风速','rolling_sum3']].head()

,风速,rolling_sum3
0,-0.855915,NaN
1,-0.851950,NaN
2,-0.847984,-2.555849
3,-0.844019,-2.543952
4,-0.838731,-2.530734


In [30]:
data_1['风速_diff2']=data.风速.diff(-2)
data_1['风速_diff1']=data.风速.diff(-1)

In [33]:
data_1[['风速','风速_diff1','风速_diff2']].head()

,风速,风速_diff1,风速_diff2
0,-0.855915,-0.003966,-0.007931
1,-0.851950,-0.003966,-0.007931
2,-0.847984,-0.003966,-0.009253
3,-0.844019,-0.005288,-0.010575
4,-0.838731,-0.005288,-0.011897


In [27]:
pd.to_datetime(data_1['时间']).dt.year

0        2017
1        2017
2        2017
3        2017
4        2017
5        2017
6        2017
7        2017
8        2017
9        2017
10       2017
11       2017
12       2017
13       2017
14       2017
15       2017
16       2017
17       2017
18       2017
19       2017
20       2017
21       2017
22       2017
23       2017
24       2017
25       2017
26       2017
27       2017
28       2017
29       2017
         ... 
78219    2019
78220    2019
78221    2019
78222    2019
78223    2019
78224    2019
78225    2019
78226    2019
78227    2019
78228    2019
78229    2019
78230    2019
78231    2019
78232    2019
78233    2019
78234    2019
78235    2019
78236    2019
78237    2019
78238    2019
78239    2019
78240    2019
78241    2019
78242    2019
78243    2019
78244    2019
78245    2019
78246    2019
78247    2019
78248    2019
Name: 时间, Length: 78249, dtype: int64

In [31]:
data_1['year'] = data_1['时间戳'].dt.year
data_1['month'] = data_1['时间戳'].dt.month
data_1['day'] = data_1['时间戳'].dt.day
data_1['hour'] = data_1['时间戳'].dt.hour
data_1['minute'] = data_1['时间戳'].dt.minute

In [28]:
data_1['风向'].value_counts()

-0.456783    25
-0.458006    23
-0.470786    20
-0.447059    20
-0.484122    20
-0.463173    20
-0.466896    20
-0.462007    20
-0.467674    20
-0.484455    19
-0.457950    19
-0.498514    19
-0.443392    19
-0.468841    19
-0.480566    19
-0.507182    19
-0.472509    19
-0.449893    19
-0.477454    18
-0.475565    18
-0.488789    18
-0.471675    18
-0.460673    18
-0.463840    18
-0.472564    17
-0.453394    17
-0.458673    17
-0.459506    17
-0.471953    17
-0.474787    17
             ..
-0.059762     1
 0.220571     1
-0.656655     1
 0.772122     1
 0.928431     1
 0.141389     1
 0.096936     1
 0.971439     1
 0.509516     1
-0.190287     1
 0.068875     1
-0.180007     1
 0.245298     1
 0.318367     1
 0.344428     1
-0.095102     1
 0.730781     1
 0.901759     1
-0.173061     1
 0.285417     1
 0.982497     1
 0.847970     1
 0.326536     1
 0.790348     1
 0.743784     1
-0.171061     1
 0.003251     1
 0.133998     1
-0.015420     1
-1.000000     1
Name: 风向, Length: 27173,

In [32]:
data_1['风向_count']= data_1['风向'].map(data_1['风向'].value_counts()).astype(int)
data_1['风向_count'] = (data_1['风向_count']-data_1['风向_count'].min()) / (data_1['风向_count'].max()-data_1['风向_count'].min())

In [33]:
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_1.groupby(['year','month','day','hour'])[item].agg({item+'时最小':'min'})
    data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day','hour'])
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_1.groupby(['year','month','day'])[item].agg({item+'日最小':'min'})
    data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day'])
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_1.groupby(['year','month'])[item].agg({item+'月最小':'min'})
    data_1 = pd.merge(data_1,t1,how='left',on=['year','month'])

E:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
E:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """
E:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [35]:
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_1.groupby(['year','month','day','hour'])[item].agg({item+'时最小':'min',item+'时最大':'max',item+'时中位数':'median',item+'时平均':'mean'})
#     data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day','hour'])

E:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [37]:
# for item in ['辐照度','风速','温度','压强','湿度']:
#     data_1[item+'时最小']=data_1.groupby(['year','month','day','hour'])[item].transform(min)
#     data_1[item+'时最小']=data_1.groupby(['year','month','day','hour'])[item].transform(min)

In [38]:
data_1['辐照度时最小'].head()

0   -1.0
1   -1.0
2   -1.0
3   -1.0
4   -1.0
Name: 辐照度时最小, dtype: float64

In [31]:
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_1.groupby(['year','month','day','hour'])[item].agg({item+'时中值':'median'})
#     data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day','hour'])
    
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_1.groupby(['year','month','day'])[item].agg({item+'日中值':'median'})
    data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day'])
    
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_1.groupby(['year','month'])[item].agg({item+'月中值':'median'})
#     data_1 = pd.merge(data_1,t1,how='left',on=['year','month'])

E:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [32]:
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_1.groupby(['year','month','day','hour'])[item].agg({item+'时最大':'max'})
    data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day','hour'])
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_1.groupby(['year','month','day'])[item].agg({item+'日最大':'max'})
#     data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day'])
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_1.groupby(['year','month'])[item].agg({item+'月最大':'max'})
#     data_1 = pd.merge(data_1,t1,how='left',on=['year','month'])
#     data_1[item+'时最大'] = data_1[item+'时最大'] - data_1[item]

E:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [33]:
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_1.groupby(['year','month','day','hour'])[item].agg('mean').reset_index().rename(columns={item:item+'时平均'})
    data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day','hour'])
    data_1[item+'时平均'] = data_1[item] - data_1[item+'时平均']
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_1.groupby(['year','month','day'])[item].agg('mean').reset_index().rename(columns={item:item+'日平均'})
#     data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day'])
#     data_1[item+'日平均'] = data_1[item] - data_1[item+'日平均']
# data_1.drop(['year'],axis=1,inplace=True)
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_1.groupby(['year','month'])[item].agg('mean').reset_index().rename(columns={item:item+'月平均'})
    data_1 = pd.merge(data_1,t1,how='left',on=['year','month'])
    data_1[item+'月平均'] = data_1[item] - data_1[item+'月平均']
data_1.drop(['year'],axis=1,inplace=True)

In [35]:
column_names = ['辐照度','温度','压强','湿度']
poly_transformer = PolynomialFeatures(degree=2,interaction_only=False,include_bias=False)
poly_data = pd.DataFrame(poly_transformer.fit_transform(data_1[column_names]),columns=poly_transformer.get_feature_names(column_names))
poly_data.drop(column_names,axis=1,inplace=True)
data_1 = pd.concat([data_1,poly_data],axis=1,sort=False)

,时间,辐照度,风速,风向,温度,湿度,压强,实际辐照度,实际功率,id,时间戳,辐照度-1,风速-1,风向-1,温度-1,压强-1,湿度-1,辐照度-2,风速-2,风向-2,温度-2,压强-2,湿度-2,辐照度+1,风速+1,风向+1,温度+1,压强+1,湿度+1,辐照度+2,风速+2,风向+2,温度+2,压强+2,湿度+2,辐照度+3,风速+3,风向+3,温度+3,压强+3,湿度+3,month,day,hour,minute,风向_count,辐照度时最小,风速时最小,温度时最小,压强时最小,湿度时最小,辐照度日最小,风速日最小,温度日最小,压强日最小,湿度日最小,辐照度月最小,风速月最小,温度月最小,压强月最小,湿度月最小,辐照度日中值,风速日中值,温度日中值,压强日中值,湿度日中值,辐照度时最大,风速时最大,温度时最大,压强时最大,湿度时最大,辐照度时平均,风速时平均,温度时平均,压强时平均,湿度时平均,辐照度月平均,风速月平均,温度月平均,压强月平均,湿度月平均,辐照度^2,辐照度 温度,辐照度 压强,辐照度 湿度,温度^2,温度 压强,温度 湿度,压强^2,压强 湿度,湿度^2
0,2017-01-01 00:15:00,-1.000000,-0.855915,0.291529,-0.459561,-0.207037,0.043513,0.0,0.039000,NaN,2017-01-01 00:15:00,-1.000000,-0.855915,0.291529,-0.459561,0.043513,-0.207037,-1.000000,-0.855915,0.291529,-0.459561,0.043513,-0.207037,-1.000000,-0.851950,0.395271,-0.462905,0.042722,-0.200263,-1.000000,-0.847984,0.497236,-0.465831,0.041930,-0.192790,-1.000000,-0.844019,0.592254,-0.469592,0.041139,-0.184763,2017,1,0,15,0.000000,-1.000000,-0.891606,-0.488819,-0.026899,-0.680585,-1.0,-0.924653,-0.642215,-0.390032,-0.859128,-1.0,-1.0,-0.868339,-0.856804,-0.965819,-1.000000,-0.626570,0.025496,0.174842,-0.236312,-1.000000,0.124917,0.364681,0.573576,0.826548,0.000000,-0.328860,-0.368007,-0.204389,-0.293057,-0.422400,-0.275695,-0.447325,-0.161854,-0.162817,1.000000,0.459561,-0.043513,0.207037,0.211196,-0.019997,0.095146,0.001893,-0.009009,0.042864
1,2017-01-01 00:30:00,-1.000000,-0.851950,0.395271,-0.462905,-0.200263,0.042722,0.0,0.039000,NaN,2017-01-01 00:30:00,-1.000000,-0.855915,0.291529,-0.459561,0.043513,-0.207037,-1.000000,-0.851950,0.395271,-0.462905,0.042722,-0.200263,-1.000000,-0.847984,0.497236,-0.465831,0.041930,-0.192790,-1.000000,-0.844019,0.592254,-0.469592,0.041139,-0.184763,-1.000000,-0.838731,0.675103,-0.473354,0.040348,-0.176346,2017,1,0,30,0.083333,-1.000000,-0.891606,-0.488819,-0.026899,-0.680585,-1.0,-0.924653,-0.642215,-0.390032,-0.859128,-1.0,-1.0,-0.868339,-0.856804,-0.965819,-1.000000,-0.626570,0.025496,0.174842,-0.236312,-1.000000,0.124917,0.364681,0.573576,0.826548,0.000000,-0.324894,-0.371351,-0.205180,-0.286282,-0.422400,-0.271729,-0.450669,-0.162645,-0.156043,1.000000,0.462905,-0.042722,0.200263,0.214281,-0.019776,0.092703,0.001825,-0.008556,0.040105
2,2017-01-01 00:45:00,-1.000000,-0.847984,0.497236,-0.465831,-0.192790,0.041930,0.0,0.042333,NaN,2017-01-01 00:45:00,-1.000000,-0.851950,0.395271,-0.462905,0.042722,-0.200263,-1.000000,-0.855915,0.291529,-0.459561,0.043513,-0.207037,-1.000000,-0.844019,0.592254,-0.469592,0.041139,-0.184763,-1.000000,-0.838731,0.675103,-0.473354,0.040348,-0.176346,-1.000000,-0.833443,0.740727,-0.477116,0.039557,-0.167724,2017,1,0,45,0.166667,-1.000000,-0.891606,-0.488819,-0.026899,-0.680585,-1.0,-0.924653,-0.642215,-0.390032,-0.859128,-1.0,-1.0,-0.868339,-0.856804,-0.965819,-1.000000,-0.626570,0.025496,0.174842,-0.236312,-1.000000,0.124917,0.364681,0.573576,0.826548,0.000000,-0.320929,-0.374276,-0.205971,-0.278809,-0.422400,-0.267764,-0.453594,-0.163436,-0.148570,1.000000,0.465831,-0.041930,0.192790,0.216998,-0.019532,0.089808,0.001758,-0.008084,0.037168
3,2017-01-01 01:00:00,-1.000000,-0.844019,0.592254,-0.469592,-0.184763,0.041139,0.0,0.042667,NaN,2017-01-01 01:00:00,-1.000000,-0.847984,0.497236,-0.465831,0.041930,-0.192790,-1.000000,-0.851950,0.395271,-0.462905,0.042722,-0.200263,-1.000000,-0.838731,0.675103,-0.473354,0.040348,-0.176346,-1.000000,-0.833443,0.740727,-0.477116,0.039557,-0.167724,-1.000000,-0.826834,0.783847,-0.481296,0.038766,-0.159020,2017,1,1,0,0.125000,-1.000000,-0.844019,-0.529363,-0.022943,-0.760670,-1.0,-0.924653,-0.642215,-0.390032,-0.859128,-1.0,-1.0,-0.868339,-0.856804,-0.965819,-1.000000,-0.626570,0.025496,0.174842,-0.236312,-1.000000,0.124917,0.323720,0.589399,0.824619,0.000000,-0.291006,-0.343208,-0.201658,-0.297312,-0.422400,-0.263798,-0.457356,-0.164227,-0.140543,1.000000,0.469592,-0.041139,0.184763,0.220517,-0.019319,0.086763,0.001692,-0.007601,0.034137
4,2017-01-01 01:15:00,-1.000000,-0.838731,0.675103,-0.473354,-0.176346,0.040348,0.0,0.042667,NaN,2017

In [37]:
def map_hour(h):
    if h < 7:
        return 1
    elif h < 10:
        return 2
    elif h < 13:
        return 3
    elif h < 16:
        return 4
    elif h < 19:
        return 5
    else:
        return 1

In [38]:
data_1['maphour'] = data_1['hour'].map(lambda x: map_hour(x))

In [60]:
X_train_1 = data_1[data_1['实际功率'].notnull()].drop(['时间','实际辐照度','实际功率','id','时间戳'],axis=1)
y_train_1 = data_1[data_1['实际功率'].notnull()]['实际功率']
X_test_1 = data_1[data_1['实际功率'].isnull()].drop(['时间','实际辐照度','实际功率','id','时间戳'],axis=1)

In [61]:
# X_train_1 = data_1[:len(train_1)].drop(['时间','实发辐照度','实际功率','id','时间戳'],axis=1)

# y_train_1 = data_1[:len(train_1)]['实际功率']

# X_test_1 = data_1[len(train_1):].drop(['时间','实发辐照度','实际功率','id','时间戳'],axis=1)

In [62]:
train_1_features = X_train_1.columns
train_1_features

Index(['辐照度', '风速', '风向', '温度', '湿度', '压强', '辐照度-1', '风速-1', '风向-1', '温度-1',
       '压强-1', '湿度-1', '辐照度-2', '风速-2', '风向-2', '温度-2', '压强-2', '湿度-2',
       '辐照度+1', '风速+1', '风向+1', '温度+1', '压强+1', '湿度+1', '辐照度+2', '风速+2',
       '风向+2', '温度+2', '压强+2', '湿度+2', '辐照度+3', '风速+3', '风向+3', '温度+3', '压强+3',
       '湿度+3', 'month', 'day', 'hour', 'minute', '风向_count', '辐照度时最小', '风速时最小',
       '温度时最小', '压强时最小', '湿度时最小', '辐照度日最小', '风速日最小', '温度日最小', '压强日最小', '湿度日最小',
       '辐照度月最小', '风速月最小', '温度月最小', '压强月最小', '湿度月最小', '辐照度日中值', '风速日中值',
       '温度日中值', '压强日中值', '湿度日中值', '辐照度时最大', '风速时最大', '温度时最大', '压强时最大', '湿度时最大',
       '辐照度时平均', '风速时平均', '温度时平均', '压强时平均', '湿度时平均', '辐照度月平均', '风速月平均',
       '温度月平均', '压强月平均', '湿度月平均', '辐照度^2', '辐照度 温度', '辐照度 压强', '辐照度 湿度',
       '温度^2', '温度 压强', '温度 湿度', '压强^2', '压强 湿度', '湿度^2', 'maphour'],
      dtype='object')

In [63]:
prediction_1 = test_1[['id']]

X_train_1_loc = X_train_1.values
X_test_1_loc = X_test_1.values
y_train_1_loc = y_train_1.values

# SKB = SelectPercentile(f_regression,percentile=95).fit(X_train_1_loc,y_train_1_loc)
# X_train_1_loc = SKB.transform(X_train_1_loc)
# X_test_1_loc = SKB.transform(X_test_1_loc)

In [64]:
len(X_train_1)

65760

In [65]:
len(X_test_1)

12489

In [66]:
len(y_train_1_loc)

65760

In [67]:
len(y_train_1_loc[y_train_1_loc>0])

33810

In [68]:
from sklearn.metrics import mean_absolute_error
def my_metric_1(y_true,y_pred): 
    y_true_1 = y_true[y_true>=10*0.03]
    y_pred_1 = y_pred[y_true>=10*0.03]
    score = mean_absolute_error(y_true_1,y_pred_1)/10.0
    return 'my_metric_1',score,False  # False表示score不是越大越好(is_bigger_better)

In [70]:

from sklearn.model_selection import KFold,StratifiedKFold
model_1 = lgb.LGBMRegressor('gbdt',num_leaves=51,max_depth=-1,learning_rate=0.05,n_estimators=100,max_bin=255,subsample_for_bin=200000,
                         objective='regression',min_split_gain=0,min_child_weight=0.001,min_child_samples=20,subsample=1,subsample_freq=1,
                         colsample_bytree=1, reg_alpha=0, reg_lambda=0, random_state=2018,n_jobs=-1)

# 10折交叉训练，构造五10个模型
skf=KFold(n_splits=10, shuffle=True, random_state=2018)
baseloss = []
loss = 0
for i, (train_index, test_index) in enumerate(skf.split(X_train_1_loc,y_train_1_loc)):
    print("Fold", i)
    lgb_model = model_1.fit(X_train_1_loc[train_index], y_train_1_loc[train_index],
                          eval_names =['train','valid'],
                          eval_set=[(X_train_1_loc[train_index], y_train_1_loc[train_index]), 
                                    (X_train_1_loc[test_index], y_train_1_loc[test_index])],
                          eval_metric=my_metric_1,
                          early_stopping_rounds=50)
    baseloss.append(lgb_model.best_score_['valid']['my_metric_1'])
    loss += lgb_model.best_score_['valid']['my_metric_1']
    test_pred= lgb_model.predict(X_test_1_loc, num_iteration=lgb_model.best_iteration_)
#     print('test mean:', test_pred.mean())
    prediction_1['predict_%s' % str(i)] = test_pred
print('my_metric_1:', baseloss, loss/10)

p = prediction_1.drop(['id'],axis=1)
prediction_1['prediction']=p.mean(axis=1)

Fold 0
[1]	train's l2: 28.0888	train's my_metric_1: 0.550913	valid's l2: 27.8602	valid's my_metric_1: 0.550277
Training until validation scores don't improve for 50 rounds.
[2]	train's l2: 25.8339	train's my_metric_1: 0.528975	valid's l2: 25.6321	valid's my_metric_1: 0.528296
[3]	train's l2: 23.8002	train's my_metric_1: 0.508576	valid's l2: 23.6149	valid's my_metric_1: 0.5078
[4]	train's l2: 21.952	train's my_metric_1: 0.489259	valid's l2: 21.8002	valid's my_metric_1: 0.488635
[5]	train's l2: 20.2827	train's my_metric_1: 0.471225	valid's l2: 20.1597	valid's my_metric_1: 0.470737
[6]	train's l2: 18.7565	train's my_metric_1: 0.45416	valid's l2: 18.6532	valid's my_metric_1: 0.453771
[7]	train's l2: 17.3881	train's my_metric_1: 0.438445	valid's l2: 17.3048	valid's my_metric_1: 0.438036
[8]	train's l2: 16.1364	train's my_metric_1: 0.423428	valid's l2: 16.0841	valid's my_metric_1: 0.423381
[9]	train's l2: 14.9943	train's my_metric_1: 0.409342	valid's l2: 14.9757	valid's my_metric_1: 0.409685

[84]	train's l2: 2.35018	train's my_metric_1: 0.171105	valid's l2: 2.83286	valid's my_metric_1: 0.186522
[85]	train's l2: 2.33736	train's my_metric_1: 0.170534	valid's l2: 2.82422	valid's my_metric_1: 0.186122
[86]	train's l2: 2.32284	train's my_metric_1: 0.169967	valid's l2: 2.81184	valid's my_metric_1: 0.185666
[87]	train's l2: 2.31076	train's my_metric_1: 0.169486	valid's l2: 2.80426	valid's my_metric_1: 0.185385
[88]	train's l2: 2.29683	train's my_metric_1: 0.168916	valid's l2: 2.79156	valid's my_metric_1: 0.184967
[89]	train's l2: 2.27906	train's my_metric_1: 0.168299	valid's l2: 2.77496	valid's my_metric_1: 0.184387
[90]	train's l2: 2.26283	train's my_metric_1: 0.16779	valid's l2: 2.75896	valid's my_metric_1: 0.183918
[91]	train's l2: 2.24806	train's my_metric_1: 0.167291	valid's l2: 2.74537	valid's my_metric_1: 0.183476
[92]	train's l2: 2.23745	train's my_metric_1: 0.166872	valid's l2: 2.73988	valid's my_metric_1: 0.183202
[93]	train's l2: 2.22148	train's my_metric_1: 0.166275	v

[47]	train's l2: 3.42166	train's my_metric_1: 0.208364	valid's l2: 3.72198	valid's my_metric_1: 0.215914
[48]	train's l2: 3.37006	train's my_metric_1: 0.206715	valid's l2: 3.67753	valid's my_metric_1: 0.214473
[49]	train's l2: 3.3187	train's my_metric_1: 0.205064	valid's l2: 3.63271	valid's my_metric_1: 0.213
[50]	train's l2: 3.27169	train's my_metric_1: 0.203466	valid's l2: 3.59049	valid's my_metric_1: 0.211612
[51]	train's l2: 3.22803	train's my_metric_1: 0.20206	valid's l2: 3.55053	valid's my_metric_1: 0.210253
[52]	train's l2: 3.18598	train's my_metric_1: 0.200711	valid's l2: 3.50932	valid's my_metric_1: 0.20902
[53]	train's l2: 3.14682	train's my_metric_1: 0.199453	valid's l2: 3.47238	valid's my_metric_1: 0.207858
[54]	train's l2: 3.10292	train's my_metric_1: 0.198054	valid's l2: 3.43345	valid's my_metric_1: 0.206651
[55]	train's l2: 3.06254	train's my_metric_1: 0.196736	valid's l2: 3.39561	valid's my_metric_1: 0.205442
[56]	train's l2: 3.02875	train's my_metric_1: 0.195621	valid'

[26]	train's l2: 5.82578	train's my_metric_1: 0.268162	valid's l2: 6.2794	valid's my_metric_1: 0.277971
[27]	train's l2: 5.61028	train's my_metric_1: 0.263586	valid's l2: 6.06328	valid's my_metric_1: 0.273396
[28]	train's l2: 5.41126	train's my_metric_1: 0.259271	valid's l2: 5.86226	valid's my_metric_1: 0.269141
[29]	train's l2: 5.22068	train's my_metric_1: 0.255205	valid's l2: 5.66836	valid's my_metric_1: 0.265086
[30]	train's l2: 5.05037	train's my_metric_1: 0.251413	valid's l2: 5.49151	valid's my_metric_1: 0.261182
[31]	train's l2: 4.88988	train's my_metric_1: 0.24768	valid's l2: 5.32798	valid's my_metric_1: 0.257452
[32]	train's l2: 4.73641	train's my_metric_1: 0.244079	valid's l2: 5.16959	valid's my_metric_1: 0.253739
[33]	train's l2: 4.59649	train's my_metric_1: 0.240861	valid's l2: 5.02764	valid's my_metric_1: 0.250568
[34]	train's l2: 4.46484	train's my_metric_1: 0.237597	valid's l2: 4.89448	valid's my_metric_1: 0.247305
[35]	train's l2: 4.34713	train's my_metric_1: 0.234615	va

[10]	train's l2: 13.9545	train's my_metric_1: 0.396314	valid's l2: 14.1998	valid's my_metric_1: 0.398562
[11]	train's l2: 13.016	train's my_metric_1: 0.383903	valid's l2: 13.2637	valid's my_metric_1: 0.386443
[12]	train's l2: 12.164	train's my_metric_1: 0.372261	valid's l2: 12.4199	valid's my_metric_1: 0.375202
[13]	train's l2: 11.4015	train's my_metric_1: 0.36158	valid's l2: 11.6662	valid's my_metric_1: 0.364681
[14]	train's l2: 10.7031	train's my_metric_1: 0.351551	valid's l2: 10.9786	valid's my_metric_1: 0.354823
[15]	train's l2: 10.0637	train's my_metric_1: 0.342154	valid's l2: 10.3446	valid's my_metric_1: 0.345617
[16]	train's l2: 9.48881	train's my_metric_1: 0.333296	valid's l2: 9.7836	valid's my_metric_1: 0.337139
[17]	train's l2: 8.95999	train's my_metric_1: 0.324935	valid's l2: 9.26783	valid's my_metric_1: 0.329045
[18]	train's l2: 8.48224	train's my_metric_1: 0.317182	valid's l2: 8.79769	valid's my_metric_1: 0.321551
[19]	train's l2: 8.0428	train's my_metric_1: 0.309717	valid

...

[57]	train's l2: 3.01319	train's my_metric_1: 0.194381	valid's l2: 3.368	valid's my_metric_1: 0.2052
[58]	train's l2: 2.98029	train's my_metric_1: 0.193339	valid's l2: 3.33779	valid's my_metric_1: 0.204255
[59]	train's l2: 2.9539	train's my_metric_1: 0.192379	valid's l2: 3.31313	valid's my_metric_1: 0.203375
[60]	train's l2: 2.91977	train's my_metric_1: 0.191156	valid's l2: 3.28337	valid's my_metric_1: 0.20238
[61]	train's l2: 2.88933	train's my_metric_1: 0.190059	valid's l2: 3.25463	valid's my_metric_1: 0.201375
[62]	train's l2: 2.85808	train's my_metric_1: 0.188903	valid's l2: 3.22575	valid's my_metric_1: 0.2003
[63]	train's l2: 2.82983	train's my_metric_1: 0.187921	valid's l2: 3.20054	valid's my_metric_1: 0.199422
[64]	train's l2: 2.80424	train's my_metric_1: 0.187092	valid's l2: 3.17889	valid's my_metric_1: 0.19869
[65]	train's l2: 2.77386	train's my_metric_1: 0.185987	valid's l2: 3.15025	valid's my_metric_1: 0.197645
[66]	train's l2: 2.74711	train's my_metric_1: 0.185138	valid's l

[38]	train's l2: 4.05059	train's my_metric_1: 0.226487	valid's l2: 4.35425	valid's my_metric_1: 0.235935
[39]	train's l2: 3.96171	train's my_metric_1: 0.224047	valid's l2: 4.26986	valid's my_metric_1: 0.233664
[40]	train's l2: 3.87773	train's my_metric_1: 0.22167	valid's l2: 4.18636	valid's my_metric_1: 0.231288
[41]	train's l2: 3.79517	train's my_metric_1: 0.219482	valid's l2: 4.10547	valid's my_metric_1: 0.229208
[42]	train's l2: 3.72193	train's my_metric_1: 0.217389	valid's l2: 4.03038	valid's my_metric_1: 0.227133
[43]	train's l2: 3.65136	train's my_metric_1: 0.215342	valid's l2: 3.96133	valid's my_metric_1: 0.225016
[44]	train's l2: 3.58751	train's my_metric_1: 0.213476	valid's l2: 3.9	valid's my_metric_1: 0.223218
[45]	train's l2: 3.52655	train's my_metric_1: 0.211632	valid's l2: 3.83656	valid's my_metric_1: 0.221399
[46]	train's l2: 3.4659	train's my_metric_1: 0.209808	valid's l2: 3.77872	valid's my_metric_1: 0.219586
[47]	train's l2: 3.40796	train's my_metric_1: 0.208107	valid'

[18]	train's l2: 8.48485	train's my_metric_1: 0.317315	valid's l2: 8.62477	valid's my_metric_1: 0.31707
[19]	train's l2: 8.0457	train's my_metric_1: 0.309863	valid's l2: 8.19797	valid's my_metric_1: 0.309897
[20]	train's l2: 7.64129	train's my_metric_1: 0.302855	valid's l2: 7.79887	valid's my_metric_1: 0.303032
[21]	train's l2: 7.27496	train's my_metric_1: 0.29643	valid's l2: 7.43897	valid's my_metric_1: 0.296937
[22]	train's l2: 6.93824	train's my_metric_1: 0.290259	valid's l2: 7.1114	valid's my_metric_1: 0.291032
[23]	train's l2: 6.63534	train's my_metric_1: 0.284602	valid's l2: 6.81299	valid's my_metric_1: 0.285637
[24]	train's l2: 6.35415	train's my_metric_1: 0.279106	valid's l2: 6.54417	valid's my_metric_1: 0.280441
[25]	train's l2: 6.08859	train's my_metric_1: 0.273914	valid's l2: 6.28004	valid's my_metric_1: 0.275284
[26]	train's l2: 5.84277	train's my_metric_1: 0.268914	valid's l2: 6.04287	valid's my_metric_1: 0.270738
[27]	train's l2: 5.62485	train's my_metric_1: 0.264321	vali

[99]	train's l2: 2.14677	train's my_metric_1: 0.163507	valid's l2: 2.60666	valid's my_metric_1: 0.177334
[100]	train's l2: 2.13575	train's my_metric_1: 0.163101	valid's l2: 2.59574	valid's my_metric_1: 0.17703
Did not meet early stopping. Best iteration is:
[100]	train's l2: 2.13575	train's my_metric_1: 0.163101	valid's l2: 2.59574	valid's my_metric_1: 0.17703
Fold 9
[1]	train's l2: 28.0766	train's my_metric_1: 0.551041	valid's l2: 27.9868	valid's my_metric_1: 0.552025
Training until validation scores don't improve for 50 rounds.
[2]	train's l2: 25.8293	train's my_metric_1: 0.529229	valid's l2: 25.7644	valid's my_metric_1: 0.530357
[3]	train's l2: 23.7972	train's my_metric_1: 0.508795	valid's l2: 23.7569	valid's my_metric_1: 0.509944
[4]	train's l2: 21.951	train's my_metric_1: 0.489568	valid's l2: 21.9329	valid's my_metric_1: 0.490744
[5]	train's l2: 20.2801	train's my_metric_1: 0.471456	valid's l2: 20.2728	valid's my_metric_1: 0.472629
[6]	train's l2: 18.7635	train's my_metric_1: 0.45

[78]	train's l2: 2.43416	train's my_metric_1: 0.17483	valid's l2: 2.85643	valid's my_metric_1: 0.186093
[79]	train's l2: 2.41612	train's my_metric_1: 0.174127	valid's l2: 2.84168	valid's my_metric_1: 0.18552
[80]	train's l2: 2.3993	train's my_metric_1: 0.173492	valid's l2: 2.82869	valid's my_metric_1: 0.185047
[81]	train's l2: 2.38187	train's my_metric_1: 0.172769	valid's l2: 2.81532	valid's my_metric_1: 0.184499
[82]	train's l2: 2.36697	train's my_metric_1: 0.17214	valid's l2: 2.80407	valid's my_metric_1: 0.184057
[83]	train's l2: 2.34972	train's my_metric_1: 0.171438	valid's l2: 2.78878	valid's my_metric_1: 0.183479
[84]	train's l2: 2.33305	train's my_metric_1: 0.17082	valid's l2: 2.77432	valid's my_metric_1: 0.182979
[85]	train's l2: 2.31762	train's my_metric_1: 0.170231	valid's l2: 2.76327	valid's my_metric_1: 0.182567
[86]	train's l2: 2.30229	train's my_metric_1: 0.169575	valid's l2: 2.75069	valid's my_metric_1: 0.182011
[87]	train's l2: 2.28692	train's my_metric_1: 0.168968	valid

In [72]:
train_features = X_train_1.columns
model_1.feature_importances_
pd.DataFrame({'features':train_features, 'imp':model_1.feature_importances_}).sort_values('imp',ascending=False)

,features,imp
68,温度时平均,256
69,压强时平均,185
37,day,175
14,风向-2,166
70,湿度时平均,157
57,风速日中值,149
48,温度日最小,148
59,压强日中值,135
50,湿度日最小,135
60,湿度日中值,129


In [73]:
prediction = pd.concat([prediction_1,prediction_2],axis=0,sort=False)
prediction = pd.concat([prediction,prediction_3],axis=0,sort=False)
prediction = pd.concat([prediction,prediction_4],axis=0,sort=False)
prediction

NameError: name 'prediction_2' is not defined

In [ ]:
now = datetime.datetime.now()
now = now.strftime('%m-%d-%H-%M')

prediction.rename(columns={'prediction':'predicition'},inplace=True)
prediction[['id', 'predicition']].to_csv("./result/baseline_%s.csv" % now, index=False)

In [37]:
#target encocer
data_1['压强_ctr']=data_1.groupby(['压强'])['实际功率'].transform('mean')

In [38]:
data_1['压强_ctr'].head()

0    4.621506
1    3.547774
2    3.450948
3    4.307555
4    4.405996
Name: 压强_ctr, dtype: float64

In [ ]:
data_1['压强_ctr']=data_1.groupby(['压强'])['label'].transform('sum')
data_1['压强_count']=data_1.groupby(['压强'])['压强'].transform('count')
data_1['压强_ctrz']=data_1['压强_ctr']/(data_1['压强_count']+0.0001)